In [10]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score
import re
import string 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier



In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/refs/heads/main/tweet_emotions.csv").drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
def clean_data(text):
  text = text.lower()
  text = text.translate(str.maketrans('','',string.punctuation))
  text = re.sub(r'\d+', '', text)
  text = text.strip()
  return text

def noise_removal(text):
  text = re.sub(r"http\S+|www\S+|https\S+",'',text)
  text = re.sub(r'[^A-Za-z\s]','',text)
  return text

def tokenization(text):
  tokens = word_tokenize(text)
  return tokens

def stopword_removal(tokens):
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  return tokens

def lemmatization(tokens):
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens


def preprocess_text(df):
  df['content'] = df['content'].apply(lambda content: clean_data(content))
  df['content'] = df['content'].apply(lambda content: noise_removal(content))
  df['content'] = df['content'].apply(lambda content:tokenization(content))
  df['content'] = df['content'].apply(lambda content: stopword_removal(content))
  df['content'] = df['content'].apply(lambda content: lemmatization(content))
  df['content'] = df['content'].apply(lambda tokens: ' '.join(tokens))
  return df 
  

In [4]:
df = preprocess_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhhwaitin call
2,sadness,funeral ceremonygloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [5]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [6]:
x= df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0,'happiness':1})
df.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_25332\1083041448.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0,'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhhwaitin call
2,0,funeral ceremonygloomy friday
6,0,sleep im thinking old friend want he married d...
8,0,charviray charlene love miss
9,0,kelcouch im sorry least friday


In [14]:
vectorizers = {
    "Bag of words ":CountVectorizer(),
    "Tfidf Vectorizer ": TfidfVectorizer()
}

algorithms = {
    'LogisticRegression':LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'RandomForest' : RandomForestClassifier(),
    'XGBoost':XGBClassifier(),
    'GradientBoosting':GradientBoostingClassifier()
    
}

In [20]:
import dagshub

dagshub.init(repo_owner='Nite2005', repo_name='mlops-mini-project', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/Nite2005/mlops-mini-project.mlflow")


mlflow.set_experiment("All experiment")

Accessing as Nite2005

Initialized MLflow to track repo "Nite2005/mlops-mini-project"

Repository Nite2005/mlops-mini-project initialized!

2025/07/19 23:39:18 INFO mlflow.tracking.fluent: Experiment with name 'All experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/38031a56ad0447ffabe817478c3a4ce8', creation_time=1752948557881, experiment_id='1', last_update_time=1752948557881, lifecycle_stage='active', name='All experiment', tags={}>

In [22]:
with mlflow.start_run(run_name = 'All Experiments') as parent_run:
    for algo_name,algorithm in algorithms.items():
        for vec_name,vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}",nested=True) as child_run:    
                x =vectorizer.fit_transform(df['content'])
                y = df['sentiment']

                x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
                model = algorithm
                model.fit(x_train,y_train)

                y_pred = model.predict(x_test)
                accuracy = accuracy_score(y_test,y_pred)
                precision = precision_score(y_test,y_pred)
                recall = recall_score(y_test,y_pred)
                f1 = f1_score(y_test,y_pred)

                mlflow.log_metric('accuracy',accuracy)
                mlflow.log_metric('precision',precision)
                mlflow.log_metric('recall',recall)
                mlflow.log_metric('f1_score',f1)

                mlflow.log_param("vectorizer",vec_name)
                mlflow.log_param("algorithm",algo_name)
                mlflow.log_param("test_size",0.2)

                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C",model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha",model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimator",model.n_estimators)
                    mlflow.log_param("learning_rate",model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators",model.n_estimators)
                    mlflow.log_param("max_depth",model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimator",model.n_estimators)
                    mlflow.log_param("learning_rate",model.learning_rate)
                    mlflow.log_param("max_depth",model.max_depth)
                
                notebook_path = 'exp1_baseline_model.ipynb'
                mlflow.log_artifact(notebook_path)



🏃 View run LogisticRegression with Bag of words  at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1/runs/99dcb4227cfd4b25b3b43dbabdbbeaf1
🧪 View experiment at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1
🏃 View run LogisticRegression with Tfidf Vectorizer  at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1/runs/40d54d6f42b74b8eb8dbf8936350790f
🧪 View experiment at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1
🏃 View run MultinomialNB with Bag of words  at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1/runs/aa2a4812b1e34f969183ce2d9fb1f0f8
🧪 View experiment at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1
🏃 View run MultinomialNB with Tfidf Vectorizer  at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/1/runs/45dabd8ebd3246f68a80afec4a8b1b5e
🧪 View experiment at: https://dagshub.com/Nite2005/mlops-mini-project.mlflo